# RAGアプリケーション構築チャレンジ



ちゅらデータの社員インタビュー記事（あるいはお好きなWebサイト）のリストを読み込んで、RAGアプリケーションを構築してみましょう

In [1]:
# 必要なライブラリをインストール
%pip install --upgrade --quiet langchain langchain-google-genai langchain-community faiss-cpu streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB

In [35]:
from google.colab import userdata
userdata.get('GEMINI_API_KEY')[:3]

'AIz'

In [3]:
######################################
#
# RAGアプリケーションを実装してみましょう
#
######################################

In [10]:
# インポート関連

## スクレイピング
import time
import requests
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

## 前処理(チャンキング)
from langchain_community.document_loaders import BSHTMLLoader
from langchain.text_splitter import CharacterTextSplitter

## RAG
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # ベクトル
from langchain_community.vectorstores import FAISS  # DB
from langchain.chains import RetrievalQA  # RAG

In [42]:
# Geminiモデルを定義
llm = ChatGoogleGenerativeAI(
    google_api_key=userdata.get('GEMINI_API_KEY'),
    model="gemini-1.5-flash-002",
    temperature=0.0,
    max_retries=2,
)

In [6]:
# Geminiに渡すプロンプトを定義する
prompt = ChatPromptTemplate.from_messages([
    ("human", "あなたは物件相談者です。入力された情報から当てはまる物件を提示してください: {user_input}"),
])

In [7]:
# WebページにアクセスしてHTMLデータをローカルに保存
# URLリストの作成
urls = [
    'https://suumo.jp/chintai/jnc_000093476941/?bc=100401392743',
    'https://suumo.jp/chintai/jnc_000093476940/?bc=100384083881',
    'https://suumo.jp/chintai/jnc_000093476951/?bc=100401374874',
    'https://suumo.jp/chintai/jnc_000076500452/?bc=100398112786',
    'https://suumo.jp/chintai/jnc_000076795791/?bc=100401046545',
    'https://suumo.jp/chintai/jnc_000093330017/?bc=100400417866',
    'https://suumo.jp/chintai/jnc_000092221976/?bc=100392416501',
    'https://suumo.jp/chintai/jnc_000093443913/?bc=100401146243',
    'https://suumo.jp/chintai/jnc_000074270541/?bc=100401170016',
    'https://suumo.jp/chintai/jnc_000083653428/?bc=100350466567'
]

In [8]:
# for文でURLごとに取得して表示
for i, url in enumerate(urls):
    response = requests.get(url)  # HTMLの取得
    content = response.text
    with open(f'./suumo_{i + 1}.html',mode='w') as fout:
        fout.write(response.text)  # 物件ごとにファイルに書き込み
    time.sleep(2)

In [9]:
# 保存されたHTMLファイルをロード
datasources = []
for i in range(len(urls)):
    file_path = f'./suumo_{i + 1}.html'

    # BSHTMLLodderを使ってHTMLファイルをロード
    loader = BSHTMLLoader(file_path=file_path)

    # ロードしたデータソースをリストに追加
    datasources.append(loader.load())
    print(f'Data from {file_path}:')
    print(datasources[i])

Data from ./suumo_1.html:
[Document(metadata={'source': './suumo_1.html', 'title': '【SUUMO】シェーネ千原II／沖縄県那覇市字小禄／奥武山公園駅の賃貸・部屋探し情報（000093476941） | 賃貸マンション・賃貸アパート'}, page_content='\n\n\n\n【SUUMO】シェーネ千原II／沖縄県那覇市字小禄／奥武山公園駅の賃貸・部屋探し情報（000093476941） | 賃貸マンション・賃貸アパート\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSUUMO(スーモ)九州・沖縄版\n\n\n\n\n\n\n全国へ\n\n\n\n\n初めての方へサイトマップお問い合わせ\n\n\n\n\nマンション・購入\n一戸建て購入・建築\n\n\n\n賃貸\nマンション購入 新築\nマンション購入 中古\n一戸建て購入・建築 新築\n一戸建て購入・建築 中古\n一戸建て購入・建築 土地\n一戸建て購入・建築 注文\nリフォーム\n売却\n相談\nお役立ち\n\n引越し見積もり\n\n\n\n\n\n\n\n\n          \xa0\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nマイリスト（物件）\nマイリスト（会社）\nマイリスト（検索条件）\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [11]:
#チャンキング作業内容
# 実際には分割せずにデータをそのまま処理
text_splitter = CharacterTextSplitter(
    separator = '',  # 区切り文字は空（つまり、分割しない）
    chunk_size = 100000000000000000,  # 非現実的に大きなサイズを指定し、分割を防止
    chunk_overlap = 0,
    length_function = len,
)

# 各データソースに対してドキュメントを作成し、分割せずにそのまま追加
docses = []
for datasource in datasources:
    print(datasource)
    # データをそのまま1つのドキュメントとして追加
    docs = text_splitter.create_documents([datasource[0].page_content])
    docses.append(docs)


[Document(metadata={'source': './suumo_1.html', 'title': '【SUUMO】シェーネ千原II／沖縄県那覇市字小禄／奥武山公園駅の賃貸・部屋探し情報（000093476941） | 賃貸マンション・賃貸アパート'}, page_content='\n\n\n\n【SUUMO】シェーネ千原II／沖縄県那覇市字小禄／奥武山公園駅の賃貸・部屋探し情報（000093476941） | 賃貸マンション・賃貸アパート\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSUUMO(スーモ)九州・沖縄版\n\n\n\n\n\n\n全国へ\n\n\n\n\n初めての方へサイトマップお問い合わせ\n\n\n\n\nマンション・購入\n一戸建て購入・建築\n\n\n\n賃貸\nマンション購入 新築\nマンション購入 中古\n一戸建て購入・建築 新築\n一戸建て購入・建築 中古\n一戸建て購入・建築 土地\n一戸建て購入・建築 注文\nリフォーム\n売却\n相談\nお役立ち\n\n引越し見積もり\n\n\n\n\n\n\n\n\n          \xa0\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nマイリスト（物件）\nマイリスト（会社）\nマイリスト（検索条件）\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n×\xa0閉じる\n\n\n\nsuumo\n\n

In [12]:
print(len(docses))

10


In [37]:
# Google Generative AIの埋め込みモデルの初期化
embeddings = GoogleGenerativeAIEmbeddings(
    google_api_key=userdata.get('GEMINI_API_KEY'),  # APIキーを取得
    model="models/embedding-001"  # 使用する埋め込みモデル
)

In [38]:
# ドキュメントをベクトルDBに追加
# FAISSベースのベクトルDBを作成し、各ドキュメントを追加する
db = None
for docs in docses:
    if db is None:
        # 最初のドキュメントセットでDBを作成
        db = FAISS.from_documents(docs, embeddings)
    else:
        # 2つ目以降のドキュメントセットは既存のDBに追加
        db.add_documents(docs)

# ベクトルDBをローカルに保存
# 'faiss_store'という名前でデータベースを保存(ここの名前を変更すると違うDBが作成できる)
db.save_local('faiss_store')


In [39]:
# DBに保存されているベクトルの総数を取得
num_vectors = len(db.index_to_docstore_id) # index_to_docstore_id は、ベクトルとそれに対応する物件データのIDを関連付ける辞書
print(f"データベースに格納されているベクトル数: {num_vectors}")

データベースに格納されているベクトル数: 10


In [43]:
# ローカルに保存されているベクトルDBをロード
db = FAISS.load_local('faiss_store', embeddings, allow_dangerous_deserialization=True)

# クエリに基づいてベクトルDBから類似度検索を行うためのretriverの作成
retriever = db.as_retriever()

# LLMを使って検索するための「質問応答システム」を作成
qa = RetrievalQA.from_chain_type(
    llm=llm,  # LLMとしてGeminiを指定
    chain_type="stuff",  # ベクトル検索の方式を指定（ここでは単純な検索("stuff")を使う）
    retriever=retriever,  # retriverの指定
    return_source_documents=True,  # 検索結果に対応する元のドキュメント（物件情報）も返す
)

In [45]:
# クエリの実行
query = "沖縄の物件を列挙してください。"
respons = qa.invoke(query)
print(respons['query'])
print(respons['result'])

沖縄の物件を列挙してください。
はい、以下の沖縄の物件を列挙します。

* アンドーバーテラス金城　サウス (那覇市金城１、小禄駅徒歩14分)
* レオネクスト那覇西高校前 (那覇市金城２、小禄駅徒歩12分)
* ゆちばな２泉崎 (那覇市泉崎１、旭橋駅徒歩6分)
* メゾン　Ｓｈｉｎｋａ (那覇市安里１、牧志駅徒歩4分)

これらの物件情報はSUUMOからの抜粋です。  それぞれの物件の詳細については、SUUMOのウェブサイトで確認することをお勧めします。



沖縄の物件を列挙してください。
はい、以下の沖縄の物件を列挙します。

* アンドーバーテラス金城　サウス (那覇市金城１、小禄駅徒歩14分)
* レオネクスト那覇西高校前 (那覇市金城２、小禄駅徒歩12分)
* ゆちばな２泉崎 (那覇市泉崎１、旭橋駅徒歩6分)
* メゾン　Ｓｈｉｎｋａ (那覇市安里１、牧志駅徒歩4分)

これらの物件情報はSUUMOからの抜粋です。  それぞれの物件の詳細については、SUUMOのウェブサイトで確認できます。



In [46]:
query2 = "那覇市で家賃6万円前後の物件を教えて"
respons2 = qa.invoke(query2)
print(respons2['query'])
print(respons2['result'])

那覇市で家賃6万円前後の物件を教えて
提示されたテキストには、那覇市で家賃6万円前後の物件が複数掲載されています。

具体的には、以下の物件が該当します。

* **メゾン　Ｓｈｉｎｋａ**: 家賃6.6万円、管理費3900円。沖縄都市モノレール牧志駅徒歩4分。
* **アンドーバーテラス金城　サウス**: 家賃6万円、管理費2200円。沖縄都市モノレール小禄駅徒歩14分。
* **ゆちばな２泉崎**: 家賃6.9万円、管理費3100円。沖縄都市モノレール旭橋駅徒歩6分。

これらの物件以外にも、テキスト中には複数の物件が掲載されていますが、家賃が6万円前後かどうかは明記されていません。  より詳細な情報や、他の物件の情報を知りたい場合は、SUUMOなどの不動産サイトで直接検索することをお勧めします。



In [47]:
query3 = "今わかる那覇市の物件を全て教えて"
respons3 = qa.invoke(query3)
print(respons3['query'])
print(respons3['result'])

今わかる那覇市の物件を全て教えて
このテキストには、那覇市にある以下の物件の情報が含まれています。

* アンドーバーテラス金城 サウス
* レオネクスト那覇西高校前
* ゆちばな２泉崎
* メゾン　Ｓｈｉｎｋａ

それぞれの物件の詳細（住所、賃料、間取り、設備など）はテキストに記載されています。  それ以外の那覇市の物件については、このテキストからは分かりません。



In [48]:
# コメントを外して実行してください
# %env GOOGLE_API_KEY={userdata.get('GEMINI_API_KEY')}

In [52]:
%%writefile app.py
from langchain.chains import RetrievalQA
from langchain.schema import (SystemMessage, HumanMessage, AIMessage)
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
import os
import streamlit as st


def load_db(embeddings):
    return FAISS.load_local('faiss_store', embeddings, allow_dangerous_deserialization=True)


def init_page():
    st.set_page_config(
        page_title='ChatGPTを活用したRAGアプリケーション',
        page_icon="🧑‍💻"
    )
    st.header('DEMO')


def main():
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001"
    )
    db = load_db(embeddings)
    init_page()

    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash-002",
        temperature=0.0,
        max_retries=2,
    )
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(),
        return_source_documents=True,
    )

    # ユーザーの入力を監視
    if "messages" not in st.session_state:
        st.session_state.messages = []
    if user_input := st.chat_input('質問を入力して下さい'):
        # 以前のチャットログを表示
        for message in st.session_state.messages:
            with st.chat_message(message["role"]):
                st.markdown(message["content"])
        with st.chat_message('user'):
            st.markdown(user_input)
        st.session_state.messages.append({"role": "user", "content": user_input})
        with st.chat_message('assistant'):
            with st.spinner('Gemini is typing ...'):
                response = qa.invoke(user_input)
            st.markdown(response['result'])
        st.session_state.messages.append({"role": "assistant", "content": response["result"]})


if __name__ == '__main__':
    main()

Overwriting app.py


In [50]:
!npm install -g localtunnel


changed 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [56]:
!streamlit run app.py & sleep 3 && npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.116.40:8501

  Stopping...
^C


もし余裕がある方がいれば・・・以下にトライしてみてください

**テキスト分割方法の工夫**  
様々なテキスト分割の手法が提供されています。ぜひ試してみましょう。  
参考: https://zenn.dev/buenotheebiten/articles/af5cfba98b1b8f

・・・いっそのこと分割せずに全部入力してみるのも面白いかも？

**System Instructionのチューニング**  
RetrievalQAの内部では、以下のようなSystem Instructionが設定されています。  
この内容を変更することで、結果がどのように変化するか確認してみましょう。
```
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know,
don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
```
変更したい場合は以下のように書けます。
```python
from langchain.prompts import PromptTemplate

prompt_template = """設定したいSystem Instructionを記載してください"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    # ここで指定する
    chain_type_kwargs={"prompt": PROMPT}
)
```